In [0]:

##(a) Fact Exposure table – main regulatory output

df_fact = spark.read.format("delta").load("/mnt/gold/mas610/fact_exposure.delta")
display(df_fact.limit(10))
df_fact.printSchema()
print(f"Total records: {df_fact.count()}")

##(b) MAS610 Report – for auditor/BI consumption

df_report = spark.read.parquet("/mnt/gold/mas610/MAS610_Report.parquet")
display(df_report.limit(10))

##(c) DQ Results – optional audit table

df_dq = spark.read.format("delta").load("/mnt/gold/mas610/dq_results.delta")
display(df_dq)

##(d) Currency Summary – aggregate validation

spark.read.csv("/mnt/gold/mas610/summary_currency.csv", header=True).display()

##Total_EAD > 0, Total_RWA > 0.

##Avg_Risk_Pct ≈ 50–100 depending on mix of secured / unsecured.

# (e) Optional Cross-Check Totals
spark.sql("""
SELECT 
  COUNT(DISTINCT loan_id) AS total_loans,
  SUM(EAD) AS total_ead,
  SUM(RWA) AS total_rwa,
  ROUND(SUM(RWA)/SUM(EAD)*100,2) AS avg_risk_weight
FROM fact_exposure
""").display()

#Confirms aggregate numbers match the summary CSV

#** Validation Narration

# “The ADF pipeline orchestrates three Databricks notebooks — Bronze, Silver, and Gold — which produce a MAS 610-compliant exposure dataset.
# After the run, I validated outputs directly in Databricks:

# fact_exposure.delta shows no null customer IDs and correct RWA calculations.

# The MAS610_Report.parquet is ready for BI consumption.

# The dq_results.delta verifies all rules passed.

# ADF Monitor confirms each activity succeeded.
# This demonstrates data quality and report accuracy for MAS 610 returns.”